Installation des librairies

In [1]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 54.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [4]:
import gensim
import gensim.downloader

1.a) Récupération du Corpus Google

In [5]:
w2v_model = gensim.downloader.load("word2vec-google-news-300")
w2v_vectors = w2v_model.wv
del w2v_model

[==================================================] 100.0% 1662.8/1662.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Save KeyedVectors on disk

In [6]:
from gensim.models import KeyedVectors

In [7]:
w2v_vectors.save('w2v_vectors.kv')

In [8]:
reloaded_w2v = KeyedVectors.load('w2v_vectors.kv', mmap='r')

1.b) Le notebook utilise 7.10Gb une fois le vecteur chargé

In [12]:
reloaded_w2v.vector_size

300

1.c) La dimension de l'espace vectoriel est 300

In [25]:
print(len(reloaded_w2v.vocab))
vocab = reloaded_w2v.vocab
words = ["shoulder", "bauble", "parrot", "nonplussed", "mischievous", "beguile", "otorhinolaryngologist", "epeolatry", "anatidaephobia"]
print([word in vocab for word in words])

3000000
[True, True, True, True, True, True, False, False, False]


1.d) La taille du vocabulaire est 3'000'000

In [28]:
reloaded_w2v.similarity("rabbit", "carrot")

0.36306435

1.e) La distance est 0.637. Elle est calculé comme 1 moins la similarité.
La similarité entre deux mot est le cosinus de l'angle entre leurs vecteurs.

In [40]:
pairs = [
         ("astrophysicist", "spaceship"),
         ("beer", "wine"),
         ("progamming", "computer"),
         ("programmer", "coder"),
         ("cat", "animal"),
         ("food", "steak"),
         ("cactus", "flower"),
         ("boy", "girl"),
         ("moon", "chair"),
]
[(w1, w2, reloaded_w2v.distance(w1,w2)) for (w1,w2) in pairs]

[('astrophysicist', 'spaceship', 0.6500389277935028),
 ('beer', 'wine', 0.3782304525375366),
 ('progamming', 'computer', 0.8665198385715485),
 ('programmer', 'coder', 0.3886645436286926),
 ('cat', 'animal', 0.40923118591308594),
 ('food', 'steak', 0.6963566839694977),
 ('cactus', 'flower', 0.5169245898723602),
 ('boy', 'girl', 0.14567279815673828),
 ('moon', 'chair', 0.9386307448148727)]

Globalement, notre intuition trouve les mots plus proches de la distance mesurée. Par exemple, on s'attend à ce que "programming" et "computer" soit relativement proche, mais ils sont en fait très éloignés, idem pour "food" et "steak".

In [42]:
opposed_pairs = [
         ("flammable", "inflammable"),
         ("accelerate", "decelerate"),
         ("free", "expensive"),
         ("dead", "alive"),
         ("strict", "chaotic"),
         ("random", "ordered"),
         ("water", "fire"),
         ("water", "land"),
]
[(w1, w2, reloaded_w2v.distance(w1,w2)) for (w1,w2) in opposed_pairs]

[('flammable', 'inflammable', 0.3406251072883606),
 ('accelerate', 'decelerate', 0.5433007180690765),
 ('free', 'expensive', 0.8415442407131195),
 ('dead', 'alive', 0.5353720188140869),
 ('strict', 'chaotic', 0.7622213512659073),
 ('random', 'ordered', 0.947737280279398),
 ('water', 'fire', 0.7728232890367508),
 ('water', 'land', 0.6596656739711761)]

On peut voir que "flammable" et "inflammable" sont vraiment très proche. Cela vient sans aucun doute du fait qu'ils sont interchangeable dans la langue oral, même si à l'origine ils sont de sens opposé. On remarque également les couples "accelerate"-"decelerate" et "dead"-"alive" qui sont de sens opposés mais relativement proche. Ceci peut venir du faire que ces mots sont souvent interchangeables dans une phrase.
C'est un défaut pour trouver des synonymes, mais une qualité pour regrouper les mots qui ont des sens similaires.

In [56]:
mm_word = "bark"
word_with_same_meanings = ["tree", "dog"]
[(w, reloaded_w2v.distance(mm_word, w)) for w in word_with_same_meanings]

[('tree', 0.5904216468334198), ('dog', 0.5834362804889679)]

Le model ne fait pas de différence entre les significations, et reste donc relativement éloignés de chacun des sens. Un humain à l'opposé associera un mot avec la signification la plus proche d'un autre, et réduira les distances entre eux.

In [77]:
from gensim.test.utils import datapath
reloaded_w2v.evaluate_word_pairs(datapath('wordsim353.tsv'))

((0.6238773466616107, 1.7963237724171284e-39),
 SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45),
 0.0)